In [1]:
import os
import datetime
import pandas as pd
import numpy as np

from keras.initializers import he_uniform
from keras.models import Sequential, load_model
from keras.optimizers import Adam, RMSprop
from keras.regularizers import l2
from keras.layers import Dense, Dropout, BatchNormalization, Activation
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler, EarlyStopping, ModelCheckpoint

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import confusion_matrix

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

seed=11
np.random.seed(seed)

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


ImportError: Traceback (most recent call last):
  File "/usr/local/lib/python3.5/dist-packages/tensorflow/python/pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "/usr/local/lib/python3.5/dist-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "/usr/local/lib/python3.5/dist-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "/usr/lib/python3.5/imp.py", line 242, in load_module
    return load_dynamic(name, filename, file)
  File "/usr/lib/python3.5/imp.py", line 342, in load_dynamic
    return _load(spec)
ImportError: libcublas.so.9.0: cannot open shared object file: No such file or directory


Failed to load the native TensorFlow runtime.

See https://www.tensorflow.org/install/install_sources#common_in

## 0. Preps

In [7]:
def sample_random_predictions(X, y):
    I = np.random.permutation(X.shape[0])[:3]

    for c, i in enumerate(I):
        img = X[i].reshape(28, 28)
        plt.subplot(131+c)
        plt.imshow(img, cmap=plt.cm.gray)
        plt.title(y[i])

In [8]:
def print_training_history(history):
    acc_list = history.history['acc']
    acc = acc_list[-1]
    best_acc_index = np.array(acc_list).argmax()
    best_acc = acc_list[best_acc_index]
    
    print('Accuracy: {:.4f} \tBest Accuracy: {:.4f} \t\t@ {} epoch'.format(acc, best_acc, best_acc_index+1))
    
    if 'val_acc' in  history.history.keys():
        val_acc_list = history.history['val_acc']
        val_acc = val_acc_list[-1]
        best_val_acc_index = np.array(val_acc_list).argmax()
        best_val_acc = val_acc_list[best_val_acc_index]
        print('Dev Accuracy: {:.4f} \tBest Dev Accuracy: {:.4f} \t@ {} epoch'.format(val_acc, best_val_acc, best_val_acc_index+1))
    

In [9]:
def plot_training_history(history):
    plt.figure(figsize=(15,4))

    plt.subplot(121)
    plt.plot(history.history['acc'], label='Training Set')
    
    if 'val_acc' in history.history.keys():
        plt.plot(history.history['val_acc'], label='Validation Set')
    plt.title('Accuracy vs Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend(loc='best')

    plt.subplot(122)
    plt.plot(history.history['loss'], label='Training Set')
    
    if 'val_loss' in history.history.keys():
        plt.plot(history.history['val_loss'], label='Validation Set')
    plt.title('Loss vs Epochs')
    plt.xlabel('Loss')
    plt.ylabel('Accuracy')
    plt.legend(loc='best')
    plt.show()

In [10]:
def plot_error_matrix(cm):
    row_sum = cm.sum(axis=1, keepdims=True)
    norm_cm = cm/row_sum
    
    np.fill_diagonal(norm_cm, 0)
    sns.heatmap(norm_cm, robust=True, fmt="f", cmap='RdBu_r', vmin=0, vmax=4)

    plt.show()

## 1. Getting Data

In [11]:
payload_dir = '../payload/'
payload_file = 'normal-payload.npz'

payload_path = os.path.join(payload_dir, payload_file)

payload_archive = np.load(payload_path)

dev_set = payload_archive['dev_set']
test_set = payload_archive['test_set']
train_dev_set = payload_archive['train_dev_set']
train_set = payload_archive['train_set']

1.1 Training Set

In [12]:
trainX, trainy = train_set[:,1:], train_set[:,0]
trainY = np_utils.to_categorical(trainy)

In [13]:
train_devX, train_devy = train_dev_set[:,1:], train_dev_set[:,0]
train_devY = np_utils.to_categorical(train_devy)

In [14]:
testX, testy = test_set[:,1:], test_set[:,0]
testY = np_utils.to_categorical(testy)

In [15]:
devX, devy = dev_set[:,1:], dev_set[:,0]
devY = np_utils.to_categorical(devy)

## 4 The Network

### 4.1 Network Architecture

Regularization Parameters

In [16]:
dropout = 0.4
lambd = 0

In [17]:
def get_model(path=None):        
    
    model = Sequential()
    
    if path:
        return load_model(path)
    
    # layer 1
    model.add(
        Dense(1024, 
        bias_initializer='zeros', kernel_initializer=he_uniform(seed), 
        kernel_regularizer=l2(lambd),
        input_dim=784)
    )
    model.add(Dropout(dropout))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

        # layer 1
    model.add(
        Dense(512, 
        bias_initializer='zeros', kernel_initializer=he_uniform(seed), 
        kernel_regularizer=l2(lambd),
        input_dim=784)
    )
    model.add(Dropout(dropout))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    
    model.add(Dense(10, activation='softmax'))
    
    return model

In [18]:
model = None#'MLPNet.44-0.09.hdf5'

if model:
    #alpha = 0.004181203352191771
    model = os.path.join('../models', model)

In [19]:
model = get_model(model)

### 4.2 Optimization Setup

4.2.1 Hyperparameters

In [20]:
alpha = 1e-3
lr_decay = 1e-2
batch_size=128
loss = 'categorical_crossentropy'
metrics = ['accuracy']

models_dir = '../models'
modelpath = os.path.join(models_dir,'MLPNet.{epoch:02d}-{val_loss:.2f}.hdf5')

4.2.2 Callbacks

In [21]:
patience=10

In [22]:
def lr_sched(epoch):
    return 1/(1+lr_decay*epoch) * alpha#*(lr_decay**epoch)

schd = LearningRateScheduler(lr_sched, verbose=1)
early_stopping = EarlyStopping(patience=patience, verbose=1)
model_checkpoint = ModelCheckpoint(filepath=modelpath, save_best_only=True, verbose=1, monitor='val_acc')

callbacks = [schd, early_stopping, model_checkpoint]

4.2.3 Adam Optimization

In [23]:
num_epoch=500

In [24]:
adam_optimizer = Adam(lr=alpha)

In [25]:
model.compile(optimizer=adam_optimizer, loss=loss, metrics=metrics)

history=model.fit(trainX, trainY, batch_size=batch_size, epochs=num_epoch, verbose=1, 
                  callbacks=callbacks, validation_split=.1)

Train on 127093 samples, validate on 14122 samples
Epoch 1/500

Epoch 00001: LearningRateScheduler reducing learning rate to 0.001.
127093/127093 [==============================] - 83s 650us/step - loss: 0.4975 - acc: 0.8394 - val_loss: 0.2140 - val_acc: 0.9340

Epoch 00001: val_acc improved from -inf to 0.93400, saving model to ../models/MLPNet.01-0.21.hdf5
Epoch 2/500

Epoch 00002: LearningRateScheduler reducing learning rate to 0.0009900990099009901.
127093/127093 [==============================] - 86s 676us/step - loss: 0.2487 - acc: 0.9220 - val_loss: 0.1573 - val_acc: 0.9482

Epoch 00002: val_acc improved from 0.93400 to 0.94817, saving model to ../models/MLPNet.02-0.16.hdf5
Epoch 3/500

Epoch 00003: LearningRateScheduler reducing learning rate to 0.000980392156862745.
127093/127093 [==============================] - 83s 651us/step - loss: 0.1936 - acc: 0.9380 - val_loss: 0.1301 - val_acc: 0.9601

Epoch 00003: val_acc improved from 0.94817 to 0.96006, saving model to ../models/ML

4.2.3 Best Model Yet

In [ ]:
print_training_history(history)

### 4.3 Training Result

In [ ]:
plot_training_history(history)



## 5. Error Analysis

In [ ]:
def pred_error(X,y):
    pred = model.predict_classes(X)
    
    cm=confusion_matrix(y_pred=pred, y_true=y)
    row_sum = np.sum(cm, axis=1, keepdims=True)
    err_matrix = cm/row_sum
    
    np.fill_diagonal(err_matrix, 0)
    sns.heatmap(err_matrix, fmt="f", vmin=0, vmax=err_matrix.max())

    
    return np.sum(pred!=y)/len(y)*100

In [ ]:
plt.figure(figsize=(15,12))

plt.subplot(221)
plt.title('training set')
train_err = pred_error(trainX, trainy)

plt.subplot(222)
plt.title('training-dev set')
train_dev_err = pred_error(train_devX, train_devy)

plt.subplot(223)
plt.title('dev set')
dev_err = pred_error(devX, devy)

plt.subplot(224)
plt.title('test set')
test_err = pred_error(testX, testy)

print('Human Level Error:\t~{:.2f} %'.format(0.00))
print('Training Error:\t\t {:.2f} %'.format(train_err))
print('Training-Dev Error:\t {:.2f} %'.format(train_dev_err))
print('Dev Error:\t\t {:.2f} %'.format(dev_err))
print('Test Error:\t\t {:.2f} %'.format(test_err))

In [ ]:
print('Human Level Error:\t~{:.2f} %'.format(0.00))
print('Training Error:\t\t {:.2f} %'.format(train_err))
print('Training-Dev Error:\t {:.2f} %'.format(train_dev_err))
print('Dev Error:\t\t {:.2f} %'.format(dev_err))
print('Test Error:\t\t {:.2f} %'.format(test_err))